# Ingest an existing Dataset or Model

In this notebook we are going to showcase how to ingest an existing dataset or model into EOTDL.

Once it is ingested, you can use it in the same way as any other dataset or model in EOTDL (exploring, staging, etc.).

## Ingesting through the CLI

The recommended way to ingest a dataset is using the CLI.

In [1]:
!eotdl datasets ingest --help

                                                                                
 Usage: eotdl datasets ingest [OPTIONS]                                         
                                                                                
 Ingest a dataset to the EOTDL.asdf                                             
                                                                                
 This command ingests the dataset to the EOTDL. The dataset must be a folder    
 with the dataset files, and at least a README.md file (and a catalog.json file 
 for Q1+). If these files are missing, the ingestion will not work. All the     
 files in the folder will be uploaded to the EOTDL.                             
                                                                                
 The following constraints apply to the dataset name:                           
 - It must be unique                                                            
 - It must be between 3 and 

In order to ingest a dataset you will need a folder in your system with the data you want to upload.

In [2]:
!ls example_data

EuroSAT-small


For this tutorial we are going to work with a subsample of the [EuroSAT](https://www.eotdl.com/datasets/EuroSAT-RGB) dataset.

In [3]:
from glob import glob 

files = glob('webinar_data/EuroSAT-small/**/*.*', recursive=True)
files

[]

A `README.md` file is required for datasets and models, containing some basic required information (dataset authors, licens, link to source and dataset name).

In [4]:
!cat example_data/EuroSAT-small/README.md

---
name: EuroSAT-small
authors: 
  - Juan B. Pedro
license: free
source: https://github.com/earthpulse/eotdl/blob/main/tutorials/notebooks/02_ingesting.ipynb
---

# EuroSAT-small

This is a small subet of the EuroSAT dataset.


The chosen name is the one that will appear in the repository, hence it must be unique, between 3 and 45 characters long and can only contain alphanumeric characters and dashes (learn more at [https://www.eotdl.com/docs/datasets/ingest](https://www.eotdl.com/docs/datasets/ingest)).

Trying to ingest a dataset without a `README.md` file will fail.

If everything is correct, the ingestion process should work.

In [8]:
!eotdl datasets ingest -p example_data/EuroSAT-small

Ingesting folder
Ingesting directory: example_data/EuroSAT-small
Ingesting files: 100%|████████████████████████████| 7/7 [00:03<00:00,  2.04it/s]


And now your dataset is avilable at EOTDL

In [9]:
!eotdl datasets list -n eurosat-small

['EuroSAT-small', 'EuroSAT-small-community-webinar', 'EuroSAT-small-bids25-v2', 'EuroSAT-small-bids25', 'EuroSAT-small-jsl2025', 'EuroSAT-small-lps25']


> Since the `EuroSAT-small` name is already taken, this process should fail for you. To solve it, just upload the dataset with a different name. However, this will polute the EOTDL with test datasets so we encourage you to try the ingestion process with a real dataset that you want to ingest (or overwrite your test dataset in the future with useful data). In any case, you can always delete the dataset from the EOTDL using the `DELETE` button in the UI.

During the ingestion process, a `catalog.parquet` file is created with STAC metadata. If your dataset already has STAC metadata (a `catalog.json` file exists at the root of the dataset), the metadata will be parsed and added to the `catalog.parquet` file. Otherwise, the `CLI` will create a STAC-compatible metadata from the directory structure.

In [10]:
import geopandas as gpd

catalog = gpd.read_parquet('example_data/EuroSAT-small/catalog.parquet')

catalog

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-12-09 14:07:41.762050,README.md,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'a6bb30a57d0f5ff0aaa65b...,[],eotdl
1,Feature,1.0.0,[],2025-12-09 14:07:41.763603,Forest/Forest_1.tif,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'f3b8b9fef6b2df6f24792e...,[],eotdl
2,Feature,1.0.0,[],2025-12-09 14:07:41.764162,Forest/Forest_2.tif,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '2e38dab64435bfbab25bab...,[],eotdl
3,Feature,1.0.0,[],2025-12-09 14:07:41.764349,Forest/Forest_3.tif,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '3e7bb982f9db5f7dabc556...,[],eotdl
4,Feature,1.0.0,[],2025-12-09 14:07:41.764543,AnnualCrop/AnnualCrop_2.tif,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'c406cb8920858b98898b9e...,[],eotdl
5,Feature,1.0.0,[],2025-12-09 14:07:41.765168,AnnualCrop/AnnualCrop_3.tif,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '59330fce6d0bf01078db3d...,[],eotdl
6,Feature,1.0.0,[],2025-12-09 14:07:41.765531,AnnualCrop/AnnualCrop_1.tif,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '63bf72ad806aa6ae313eaf...,[],eotdl


Let's now ingest the model that we trained in the previous notebook.

In [12]:
!ls outputs/*.onnx

outputs/model.onnx


First, we need to create the `README.md` file.

In [13]:
text = """---
name: EuroSAT-RGB-tutorial
authors: 
  - Juan B. Pedro
license: open
source: https://github.com/earthpulse/eotdl/blob/develop/tutorials/workshops/community_webinar/02_training.ipynb
---

# EuroSAT-RGB-tutorial

This is a toy model trained with the EuroSAT dataset for the community webinar.
"""

with open('outputs/README.md', 'w') as outfile:
	outfile.write(text)

Now we can ingest the model to EOTDL

In [14]:
!eotdl models ingest -p outputs

Ingesting directory: outputs
Ingesting files: 100%|████████████████████████████| 2/2 [00:02<00:00,  1.07s/it]


In [15]:
!eotdl models list -n tutorial

['EuroSAT-RGB-tutorial']


## Versioning

By default, every time you re-upload a dataset or model a new version is created. We apply versioning at dataset/model and file level, meaning only new or modified files will be uploaded in future re-uploads, downloading the appropriate files for each version.

In [16]:
!echo "hello" > example_data/EuroSAT-small/hello.txt
!eotdl datasets ingest -p example_data/EuroSAT-small/

Ingesting folder
Ingesting directory: example_data/EuroSAT-small
Ingesting files: 100%|████████████████████████████| 8/8 [00:01<00:00,  4.29it/s]
A new version was created, your dataset has changed.
Num changes: 1


When you stage a dataset, the latest version is used by default.

In [17]:
!eotdl datasets get EuroSAT-small -f

Data available at /Users/juan/.cache/eotdl/datasets/EuroSAT-small


However, you can specify the version

In [18]:
!eotdl datasets get EuroSAT-small -v 1 -f

Data available at /Users/juan/.cache/eotdl/datasets/EuroSAT-small


In [19]:
!ls $HOME/.cache/eotdl/datasets/EuroSAT-small

AnnualCrop         README.md          catalog.v2.parquet
Forest             catalog.v1.parquet


You can explore the different versions in the user interface.

## Ingesting through the Library

You can ingest datasets and models using the library

In [20]:
from eotdl.datasets import ingest_dataset

try:
    ingest_dataset("example_data/EuroSAT-small")
except Exception as e:
    print(e)

Ingesting folder
Ingesting directory: example_data/EuroSAT-small


Ingesting files: 100%|██████████| 8/8 [00:01<00:00,  5.55it/s]


No new version was created, your dataset has not changed.


In [21]:
from eotdl.models import ingest_model

try:
    ingest_model("outputs")
except Exception as e:
    print(e)

Ingesting directory: outputs


Ingesting files: 100%|██████████| 2/2 [00:00<00:00,  5.12it/s]

No new version was created, your dataset has not changed.
